In [1]:
### import the following libraries
import sys
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
from PIL import Image
import os
import warnings
import tifffile as tiff

warnings.filterwarnings('ignore')

In [2]:
### directory & filepaths
data_dir = '/home/shamini/data1/data_orig/data/spatial/xenium/10xGenomics/cell_seg_brain_cancer/'
out = '/home/shamini/data/projects/spatial_workshop/out/module6/'
os.makedirs(out+'script05_figures/', exist_ok=True)

In [3]:
### attach system files for function import
sys.path.append('/home/shamini/data/projects/spatial_workshop/')
sys.path.append('/home/shamini/data/projects/spatial_workshop/Banksy_py')

import pre_processing_fnc as ppf

In [4]:
adata = sc.read_10x_h5(data_dir+'out/cell_feature_matrix.h5')
df_cell = pd.read_parquet(data_dir+'out/cells.parquet')
df_cell.set_index('cell_id', inplace=True)
df_cell.reindex(adata.obs.index)
adata.obs = df_cell
### process the adata object
sc.pp.filter_cells(adata, min_counts=9)
sc.pp.filter_genes(adata, min_cells=3)
### reduce adata size to fit the cropped image coordinates
#adata = adata[adata.obs.index.isin(df_cells_subset['cell_id']),:]
adata

AnnData object with n_obs × n_vars = 815294 × 480
    obs: 'x_centroid', 'y_centroid', 'transcript_counts', 'control_probe_counts', 'control_codeword_counts', 'unassigned_codeword_counts', 'deprecated_codeword_counts', 'total_counts', 'cell_area', 'nucleus_area', 'n_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells'

In [5]:
import umap

sc.experimental.pp.normalize_pearson_residuals(adata)
sc.pp.pca(adata)
sc.pp.neighbors(adata, n_neighbors=25)


reducer = umap.UMAP()
embedding = reducer.fit_transform(adata.obsm['X_pca'])

adata.obsm['X_umap'] = embedding

adata.uns['spatial'] = adata.obs[['x_centroid', 'y_centroid']]
adata.write_h5ad(out+'adata.h5ad')
